In [35]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [36]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [37]:
Telus_Customer_details='''



with ADC_customer_base as(

SELECT distinct customer_id,dealer_customer_id,dealer_name,account_type_name,customer_type_name
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) = '2022-08-31'
--and account_type_name!='Standalone'
--and customer_type_name='Customer'
and dealer_name='TELUS Communications Inc.'
QUALIFY ROW_NUMBER() OVER (PARTITION BY dealer_customer_id ORDER BY last_updt_ts DESC) = 1
--order by dealer_name,join_date,customer_id
)
,

Telus_customers as
(
select bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = "2022-08-31" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id

)


,Telus_internet_customers as 
(

select distinct (customer_id) as cust_id_telus,Telus_internet_flag from `divgpras-pr-579355.ADC_updated.Telus_internet_31Aug2022`

)


, Telus_SMHM_Deact_Sep2022 as 


(


select distinct(CAST (CUST_ID as STRING)) as CUST_ID ,Deact_Flag as Telus_Churn_Flag  

from `divgpras-pr-579355.ADC_updated.Telus_SMHM_Sep2022_Deacts`


)

select * from ADC_customer_base as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id
left join Telus_internet_customers
on ADC.customer_id=Telus_internet_customers.cust_id_telus
left join Telus_SMHM_Deact_Sep2022
on ADC.dealer_customer_id=Telus_SMHM_Deact_Sep2022.CUST_ID

'''

In [46]:
Telus_Customer_details_Nov2022='''



with ADC_customer_base as(

SELECT distinct customer_id,dealer_customer_id,dealer_name,account_type_name,customer_type_name
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where 
date(last_updt_ts) = '2022-11-30'
--and account_type_name!='Standalone'
--and customer_type_name='Customer'
and dealer_name='TELUS Communications Inc.'
QUALIFY ROW_NUMBER() OVER (PARTITION BY dealer_customer_id ORDER BY last_updt_ts DESC) = 1
--order by dealer_name,join_date,customer_id
)
,

Telus_customers as
(
select bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = "2022-11-30" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY prod_instnc_ts DESC) = 1
order by cust_bus_cust_id

)





, Telus_SMHM_Deact_Nov2022 as 


(


select distinct(CAST (CUST_ID as STRING)) as CUST_ID ,1 as Telus_Churn_Flag  

from `divgpras-pr-579355.ADC_updated.Telus_SMHM_Nov2022_Deacts`


)

select * from ADC_customer_base as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id
left join Telus_SMHM_Deact_Nov2022
on ADC.dealer_customer_id=Telus_SMHM_Deact_Nov2022.CUST_ID

'''

In [47]:
Data_Nov22=extract_bq_data(bq_client, sql=Telus_Customer_details_Nov2022)

In [48]:
Data_Nov22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337944 entries, 0 to 337943
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   customer_id             337944 non-null  Int64 
 1   dealer_customer_id      337944 non-null  object
 2   dealer_name             337944 non-null  object
 3   account_type_name       337944 non-null  object
 4   customer_type_name      337944 non-null  object
 5   bacct_bus_bacct_num     337944 non-null  Int64 
 6   bacct_billg_acct_id     337944 non-null  Int64 
 7   cust_bus_cust_id        337944 non-null  object
 8   pi_prod_instnc_typ_cd   337944 non-null  object
 9   pi_prod_instnc_stat_ts  337944 non-null  object
 10  prod_instnc_ts          337944 non-null  object
 11  contract_end_date       337944 non-null  object
dtypes: Int64(3), object(9)
memory usage: 31.9+ MB


In [51]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC_updated.ADC_Telus_Customers_Nov22'

bq_table_instance= bq_client.load_table_from_dataframe(Data_Nov22, Table_BQ,job_config=config)

In [50]:
Data_Nov22['Telus_Churn_Flag'].sum()

KeyError: 'Telus_Churn_Flag'

In [15]:
Data_Sep2022=extract_bq_data(bq_client, sql=Telus_Customer_details)

In [16]:
Data_Sep2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305606 entries, 0 to 305605
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   customer_id             305606 non-null  Int64 
 1   dealer_customer_id      305606 non-null  object
 2   dealer_name             305606 non-null  object
 3   account_type_name       305606 non-null  object
 4   customer_type_name      305606 non-null  object
 5   bacct_bus_bacct_num     305606 non-null  Int64 
 6   bacct_billg_acct_id     305606 non-null  Int64 
 7   cust_bus_cust_id        305606 non-null  object
 8   pi_prod_instnc_typ_cd   305606 non-null  object
 9   pi_prod_instnc_stat_ts  305606 non-null  object
 10  prod_instnc_ts          305606 non-null  object
 11  contract_end_date       305606 non-null  object
 12  cust_id_telus           216722 non-null  Int64 
 13  Telus_internet_flag     216722 non-null  Int64 
 14  CUST_ID                 3207 non-nul

In [17]:
Data_Sep2022['Telus_Churn_Flag'].sum()

3207

In [18]:
Data_Sep2022['Telus_internet_flag'].sum()

216722

In [336]:
Data_Sep2022['Telus_Churn_Flag'].fillna(0,inplace=True)

In [337]:
Data_Sep2022['Telus_internet_flag'].fillna(0,inplace=True)

In [338]:
Data_Sep2022['Telus_Churn_Flag'].value_counts(normalize=True)*100

0    98.95061
1     1.04939
Name: Telus_Churn_Flag, dtype: Float64

In [339]:
Data_Sep2022['dealer_customer_id'].isna().sum()

0

In [340]:
Data_Sep2022['customer_id'].value_counts()

8254474     1
13560457    1
8755605     1
13558223    1
9421904     1
           ..
6261724     1
7269798     1
13702410    1
13880864    1
9236191     1
Name: customer_id, Length: 305606, dtype: Int64

In [341]:
Data_Sep2022['dealer_customer_id'].value_counts()

100020154    1
19508239     1
19375756     1
19316525     1
19253502     1
            ..
104461496    1
104439987    1
104379422    1
104351220    1
99977803     1
Name: dealer_customer_id, Length: 305606, dtype: int64

In [342]:
Data_Sep2022['bacct_bus_bacct_num'].isna().sum()*100/Data_Sep2022.shape[0]

0.0

In [343]:
pd.DataFrame(Data_Sep2022.groupby(['dealer_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,dealer_name,Customer_count,Churn_total,Churn_rate
0,TELUS Communications Inc.,305606,3207,1.04939


In [344]:
pd.DataFrame(Data_Sep2022.groupby(['account_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,account_type_name,Customer_count,Churn_total,Churn_rate
0,Awareness and Automation,70993,768,1.081797
1,Security System,190276,1464,0.769409
2,Standalone,44337,975,2.199066


In [345]:
pd.DataFrame(Data_Sep2022.groupby(['customer_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,customer_type_name,Customer_count,Churn_total,Churn_rate
0,Commitment,3310,203,6.132931
1,Customer,302194,3004,0.994063
2,Demo - Normal,99,0,0.000000
3,From Recycle,3,0,0.000000


In [346]:
pd.DataFrame(Data_Sep2022.groupby(['customer_type_name','account_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Telus_matched_customers=('cust_bus_cust_id','nunique'),
    Telus_internet_customers=('Telus_internet_flag',lambda x: x.sum()),
    Telus_internet_penetration=('Telus_internet_flag',lambda x: x.mean()*100),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,customer_type_name,account_type_name,Customer_count,Telus_matched_customers,Telus_internet_customers,Telus_internet_penetration,Churn_total,Churn_rate
0,Commitment,Awareness and Automation,2037,2037,1791,87.923417,118,5.792833
1,Commitment,Security System,1261,1261,673,53.370341,84,6.661380
2,Commitment,Standalone,12,12,11,91.666667,1,8.333333
3,Customer,Awareness and Automation,68956,68956,61859,89.707930,650,0.942630
4,Customer,Security System,188913,188913,120861,63.977069,1380,0.730495
5,Customer,Standalone,44325,44325,31451,70.955443,974,2.197406
6,Demo - Normal,Security System,99,99,76,76.767677,0,0.000000
7,From Recycle,Security System,3,3,0,0.000000,0,0.000000


In [347]:
# Data_Sep2022[Data_Sep2022['account_type_name']=='Standalone'][['customer_id','cust_bus_cust_id','account_type_name']]

In [348]:
661+1431

2092

In [349]:
44490/44892

0.9910451750868752

In [350]:
Best_Practices_Query= '''

select * from `divgpras-pr-579355.ADC_updated.Best_Practices_Customers_latest`

'''

In [351]:
Best_Practices_DF=extract_bq_data(bq_client, sql=Best_Practices_Query)

In [352]:
Best_Practices_DF.head()

,customer_id,Best_partices_1,Best_partices_2,Best_partices_3,Best_partices_4,Best_partices_5,Best_partices_6,Best_Practice_All,Best_Practice_All_flag
0,11254267,1,1,0,1,1,0,4,0
1,10445690,1,1,0,1,1,0,4,0
2,6579995,1,1,0,1,1,0,4,0
3,11253105,1,1,0,1,1,0,4,0
4,11253845,1,1,0,1,1,0,4,0


In [353]:
Best_Practices_DF['Best_partices_5'].value_counts(normalize=True)*100

1    68.015069
0    31.984931
Name: Best_partices_5, dtype: Float64

In [354]:
Arming_Query=  '''

with date_sql as (
select  count(distinct date(date)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)< '2022-09-01'and date(date)>= '2022-08-01'
)

select id_cust as customer_id,sum(count_arm_commands) as sum_arm_commands,sum(count_disarm_commands) as sum_disarm_commands,count (distinct date(date)) as number_days_arming_disarming, (select count_of_dates from date_sql ) as count_of_dates_arming
from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)< '2022-09-01' and date(date)>= '2022-08-01'
group by customer_id
order by number_days_arming_disarming desc

'''

In [355]:
Arming_DF=extract_bq_data(bq_client, sql=Arming_Query)

In [356]:
Arming_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309292 entries, 0 to 309291
Data columns (total 5 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   customer_id                   309292 non-null  Int64
 1   sum_arm_commands              309292 non-null  Int64
 2   sum_disarm_commands           309292 non-null  Int64
 3   number_days_arming_disarming  309292 non-null  Int64
 4   count_of_dates_arming         309292 non-null  Int64
dtypes: Int64(5)
memory usage: 13.3 MB


In [357]:

max_number_arming_dates=Arming_DF['count_of_dates_arming'].max()
Arming_DF['Arming_Consistency']=Arming_DF['number_days_arming_disarming'].apply(lambda x: x*100/max_number_arming_dates)


In [358]:
max_number_arming_dates

18

In [359]:
Arming_DF['Arming_Flag']=1

In [360]:
Login_SQL='''

with date_sql as (
select  count(distinct date(login_dt_utc)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< '2022-09-01' and date(login_dt_utc)>= '2022-08-01'

)



select customer_id,sum(login_count) as sum_login_count,count (distinct date(login_dt_utc)) as number_of_login_days, (select count_of_dates from date_sql ) as count_of_dates_logins
from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< '2022-09-01' and date(login_dt_utc)>= '2022-08-01'
group by customer_id
order by number_of_login_days desc


'''

In [361]:
Login_DF=extract_bq_data(bq_client, sql=Login_SQL)

In [362]:
max_number_login_dates=Login_DF['count_of_dates_logins'].max()
Login_DF['Login_Consistency']=Login_DF['number_of_login_days'].apply(lambda x: x*100/max_number_login_dates)

In [363]:
Camera_TC_issue_query='''

select dealer_name,customer_id,unit_id,device_id,trouble_condition_name,trouble_condition_group_id,trouble_condition_group_desc, start_date_utc,end_date_utc
from `cio-datahub-enterprise-pr-183a.src_adc.bq_troublecondition_data`
where trouble_condition_start_ind='Y'and trouble_condition_closed_ind='Y' 
and date(start_date_utc) >= '2022-06-01' and date(start_date_utc) <'2022-09-01' 
and trouble_condition_name in ('CameraNotReachable','CameraNotCommunicating','BroadbandCommFailure')


'''

In [364]:
Camera_TC= extract_bq_data(bq_client, sql=Camera_TC_issue_query)

In [365]:
Camera_TC['count_instance']=1

TC_pivot_table=pd.pivot_table(Camera_TC, values=['count_instance'], index=['customer_id'], columns='trouble_condition_name',
                          aggfunc={'count_instance': np.sum})

In [366]:
TC_pivot_table=TC_pivot_table.reset_index()

In [367]:
TC_pivot_table.head()

customer_id       count_instance  \
trouble_condition_name             BroadbandCommFailure   
0                           213860                  2.0   
1                           229153                  NaN   
2                           287850                 20.0   
3                           292877                  3.0   
4                           299100                131.0   

                                                                  
trouble_condition_name CameraNotCommunicating CameraNotReachable  
0                                         NaN                NaN  
1                                         NaN                1.0  
2                                         NaN                NaN  
3                                         NaN                NaN  
4                                         NaN                NaN

In [368]:
TC_pivot_table.columns=['customer_id','BroadbandCommFailure_count','CameraNotCommunicating_count','CameraNotReachable_count']

In [369]:
TC_pivot_table.fillna(0,inplace=True)

In [370]:
TC_pivot_table['BroadbandCommFailure_Flag']=TC_pivot_table['BroadbandCommFailure_count'].apply(lambda x: 1 if x>0 else 0)
TC_pivot_table['CameraNotCommunicating_Flag']=TC_pivot_table['CameraNotCommunicating_count'].apply(lambda x: 1 if x>0 else 0)
TC_pivot_table['CameraNotReachable_Flag']=TC_pivot_table['CameraNotReachable_count'].apply(lambda x: 1 if x>0 else 0)


In [371]:
Merge_DF=Data_Sep2022.merge(Best_Practices_DF,on='customer_id',how='left')

In [372]:
Merge_DF_1=Merge_DF.merge(Arming_DF,on='customer_id',how='left')

In [373]:
Merge_DF_2=Merge_DF_1.merge(Login_DF,on='customer_id',how='left')

In [374]:
Merge_DF_3=Merge_DF_2.merge(TC_pivot_table,on='customer_id',how='left')

In [375]:
Merge_DF_3['Best_Practice_All_flag'].isna().sum()

6083

In [376]:
Merge_DF_3['Arming_Consistency'].isna().sum()

195849

In [377]:
Merge_DF_3['Login_Consistency'].isna().sum()

305606

In [378]:
Merge_DF_3.fillna(0,inplace=True)

In [379]:
Merge_DF_3['Login_Consistency'].value_counts()

0.0    305606
Name: Login_Consistency, dtype: int64

In [380]:
def Segment_making(row
                   ,min_login_consistency_threshold=50
                   ,min_arming_consistency_threshold=50
                   ,max_login_consistency_threshold=50
                   ,max_arming_consistency_threshold=50):
    
    if row['Best_Practice_All']>=5 and row['Login_Consistency']>=max_login_consistency_threshold  and row['Arming_Consistency']>=max_arming_consistency_threshold:
        return 'Heavy_User'
    elif row['Login_Consistency']>=max_login_consistency_threshold  and row['Arming_Consistency']< min_arming_consistency_threshold:
        return 'Home_automation_Savvy'
    elif row['Login_Consistency']< min_login_consistency_threshold  and row['Arming_Consistency']>=max_login_consistency_threshold:
        return 'Old_Fashion'
    elif row['Login_Consistency']==0  and row['Arming_Consistency']==0:
        return 'Disengaged'
    else:
        return "Moderate_Users"

In [381]:
Merge_DF_3['Segment']=Merge_DF_3.apply(Segment_making,axis=1)

In [382]:
pd.DataFrame(Merge_DF_3.groupby(['Segment']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Best_practices_1=('Best_partices_1','sum'),
     Login_conis=('Login_Consistency','mean'),
    # Telus_internet_customers=('Telus_internet_flag',lambda x: x.sum()),
    # Telus_internet_penetration=('Telus_internet_flag',lambda x: x.mean()*100),
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Segment,Customer_count,Best_practices_1,Login_conis
0,Disengaged,195849,161115,0.0
1,Moderate_Users,45110,39984,0.0
2,Old_Fashion,64647,61517,0.0


In [383]:
Merge_DF_3['Segment'].value_counts(normalize=True)*100

Disengaged        64.085456
Old_Fashion       21.153708
Moderate_Users    14.760836
Name: Segment, dtype: float64

In [384]:
Merge_DF_3['Segment'].value_counts()

Disengaged        195849
Old_Fashion        64647
Moderate_Users     45110
Name: Segment, dtype: int64

In [385]:
Merge_DF_3['Login_Consistency'].value_counts(normalize=True)*100

0.0    100.0
Name: Login_Consistency, dtype: float64

In [386]:
Merge_DF_3['Arming_Consistency'].value_counts(normalize=True)*100

0.000000      64.085456
100.000000     7.027022
5.555556       4.622619
94.444444      3.086981
11.111111      2.783322
88.888889      2.161607
16.666667      1.753238
83.333333      1.720189
77.777778      1.509133
22.222222      1.409331
72.222222      1.260774
61.111111      1.180278
27.777778      1.179623
66.666667      1.148538
55.555556      1.089311
33.333333      1.051354
38.888889      1.004234
50.000000      0.969876
44.444444      0.957115
Name: Arming_Consistency, dtype: float64

In [387]:
Merge_DF_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305606 entries, 0 to 305605
Data columns (total 41 columns):
 #   Column                        Non-Null Count   Dtype              
---  ------                        --------------   -----              
 0   customer_id                   305606 non-null  Int64              
 1   dealer_customer_id            305606 non-null  object             
 2   dealer_name                   305606 non-null  object             
 3   account_type_name             305606 non-null  object             
 4   customer_type_name            305606 non-null  object             
 5   bacct_bus_bacct_num           305606 non-null  Int64              
 6   bacct_billg_acct_id           305606 non-null  Int64              
 7   cust_bus_cust_id              305606 non-null  object             
 8   pi_prod_instnc_typ_cd         305606 non-null  object             
 9   pi_prod_instnc_stat_ts        305606 non-null  datetime64[ns]     
 10  prod_instnc_ts      

In [388]:
Merge_DF_3['Best_partices_3'].value_counts(normalize=True)*100

0    59.430116
1    40.569884
Name: Best_partices_3, dtype: Float64

In [389]:
pd.DataFrame(Merge_DF_3.groupby(['Segment']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Best_practices_3=('Best_partices_3','sum'),
     Best_pract_Arming_per=('Best_partices_3','mean'),
    # Telus_internet_customers=('Telus_internet_flag',lambda x: x.sum()),
    # Telus_internet_penetration=('Telus_internet_flag',lambda x: x.mean()*100),
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Segment,Customer_count,Best_practices_3,Best_pract_Arming_per
0,Disengaged,195849,52074,0.265889
1,Moderate_Users,45110,28947,0.641698
2,Old_Fashion,64647,42963,0.664578


In [390]:
Merge_DF_3['contract_end_date'].value_counts()

9999-12-31 00:00:00    27903
2024-06-24 00:00:00      535
2025-08-26 00:00:00      528
2025-08-18 00:00:00      528
2025-08-25 00:00:00      516
                       ...  
2018-08-18 00:00:00        1
2018-12-04 00:00:00        1
2013-10-26 00:00:00        1
2009-02-22 00:00:00        1
2017-07-21 00:00:00        1
Name: contract_end_date, Length: 4232, dtype: int64

In [391]:
# Merge_DF_2[Merge_DF_2['contract_end_date']]

In [392]:
Merge_DF_3['Camera_TC_condition_count']=Merge_DF_3.apply(lambda x: x['BroadbandCommFailure_count']+x['CameraNotCommunicating_count']+x['CameraNotReachable_count'],axis=1)

In [393]:
Merge_DF_3['Camera_TC_condition_count'].value_counts()

0.0       255433
1.0        21956
2.0         8105
3.0         4563
4.0         3301
           ...  
901.0          1
485.0          1
881.0          1
1243.0         1
1609.0         1
Name: Camera_TC_condition_count, Length: 721, dtype: int64

In [394]:
Merge_DF_3['Telus_Churn_Flag'].value_counts(normalize=True)*100

0    98.95061
1     1.04939
Name: Telus_Churn_Flag, dtype: Float64

In [395]:
Merge_DF_3.to_csv('GTM_full_data_Sep2022.csv',index=False)

In [253]:
# GTM_data_4=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_partices_4']==1) & (Merge_DF_3['Best_partices_1']==0) & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))] 
# GTM_data_5=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.')& (Merge_DF_3['Best_partices_4']==1) & (Merge_DF_3['Best_Practice_All']<3) & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))] 
# GTM_data_6=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.')& (Merge_DF_3['Best_partices_4']==1) & (Merge_DF_3['Camera_TC_condition_count']>3) & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))] 
GTM_data_7=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.')& (Merge_DF_3['Best_partices_5']==1)  & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))]

In [255]:
GTM_data_7.shape

(180275, 42)

In [256]:
GTM_data_7.to_csv('GTM_data_HomeAutodev.csv',index=False)

In [235]:
# GTM_dis_data_1=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_partices_1']==0) & (Merge_DF_3['Segment']=='Disengaged')] 
# GTM_dis_data_2=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_Practice_All']<3) & (Merge_DF_3['Segment']=='Disengaged')] 
# GTM_dis_data_3=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Camera_TC_condition_count']>3) & (Merge_DF_3['Segment']=='Disengaged')] 


In [236]:
# GTM_dis_data_2['Telus_Churn_Flag'].value_counts(normalize=True)*100

In [237]:
# GTM_dis_data_3['Telus_Churn_Flag'].value_counts(normalize=True)*100

In [238]:
# GTM_dis_data_1.to_csv('GTM_data_NoEV_disengaged.csv',index=False)

In [239]:
# GTM_dis_data_2.to_csv('GTM_data_BPlessthan3_disengaged.csv',index=False)

In [240]:
# GTM_dis_data_3.to_csv('GTM_data_CameraTC_grtr_than3_disengaged.csv',index=False)

In [241]:
# GTM_Mod_data_1=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_partices_1']==0) & (Merge_DF_3['Segment']=='Moderate_Users')] 
# GTM_Mod_data_2=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_Practice_All']<3) & (Merge_DF_3['Segment']=='Moderate_Users')] 
# GTM_Mod_data_3=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Camera_TC_condition_count']>3) & (Merge_DF_3['Segment']=='Moderate_Users')]

In [242]:
# GTM_Mod_data_1.to_csv('GTM_data_NoEV_Moderate.csv',index=False)
# GTM_Mod_data_2.to_csv('GTM_data_BPlessthan3_Moderate.csv',index=False)
# GTM_Mod_data_3.to_csv('GTM_data_CameraTC_grtr_than3_Moderate.csv',index=False)